In [1]:
import airbyte_api as ab

In [2]:
from airbyte_api import api, models
from airbyte_api.utils import RetryConfig, BackoffStrategy
from os import getenv

## Airbyte Credentials

In [3]:
api_url = getenv("AIRBYTE_API_URL", "http://localhost:8000/api/public/v1")

In [4]:
client_id = getenv("AIRBYTE_CLIENT_ID", None)

In [5]:
client_secret = getenv("AIRBYTE_CLIENT_SECRET", None)

## Airbyte API

In [6]:
client = ab.AirbyteAPI(
    server_url=api_url,
    security=models.Security(client_credentials=models.SchemeClientCredentials(
        client_id=client_id,
        client_secret=client_secret,
        token_url=api_url + "/applications/token"
    )),
    retry_config=RetryConfig(
        strategy="exponential-backoff",
        retry_connection_errors=True,
        backoff=BackoffStrategy(
            initial_interval=3,
            max_interval=30,
            max_elapsed_time=600,
            exponent=2
        )
    )
)

### List Connections

In [7]:
connections_resp = (client
    .connections
    .list_connections(api.ListConnectionsRequest(
        limit=10, 
        offset=0, 
        include_deleted=False
    ))
).connections_response

In [8]:
print(connections_resp)

ConnectionsResponse(data=[ConnectionResponse(configurations=StreamConfigurations(streams=[StreamConfiguration(name='items', cursor_field=['published'], destination_object_name=None, include_files=False, mappers=[], namespace=None, primary_key=[['guid']], selected_fields=[], sync_mode=<ConnectionSyncModeEnum.INCREMENTAL_DEDUPED_HISTORY: 'incremental_deduped_history'>)]), connection_id='b7715560-ec4e-44e8-8d64-830f6463e341', created_at=1768460945, destination_id='14e21b21-85d5-46b8-916a-df3c087ac4fe', name='hackernews-rss-newest → bigquery-hackernews', schedule=ConnectionScheduleResponse(schedule_type=<ScheduleTypeWithBasicEnum.MANUAL: 'manual'>, basic_timing=None, cron_expression=None), source_id='d73fb160-9fbd-457e-891e-22d5ac5b5aee', status=<ConnectionStatusEnum.ACTIVE: 'active'>, tags=[], workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', namespace_definition=<NamespaceDefinitionEnum.DESTINATION: 'destination'>, namespace_format=None, non_breaking_schema_updates_behavior=<NonBreaki

In [9]:
connections = connections_resp.data

In [10]:
for conn in connections:
    print(conn)
    print()

ConnectionResponse(configurations=StreamConfigurations(streams=[StreamConfiguration(name='items', cursor_field=['published'], destination_object_name=None, include_files=False, mappers=[], namespace=None, primary_key=[['guid']], selected_fields=[], sync_mode=<ConnectionSyncModeEnum.INCREMENTAL_DEDUPED_HISTORY: 'incremental_deduped_history'>)]), connection_id='b7715560-ec4e-44e8-8d64-830f6463e341', created_at=1768460945, destination_id='14e21b21-85d5-46b8-916a-df3c087ac4fe', name='hackernews-rss-newest → bigquery-hackernews', schedule=ConnectionScheduleResponse(schedule_type=<ScheduleTypeWithBasicEnum.MANUAL: 'manual'>, basic_timing=None, cron_expression=None), source_id='d73fb160-9fbd-457e-891e-22d5ac5b5aee', status=<ConnectionStatusEnum.ACTIVE: 'active'>, tags=[], workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', namespace_definition=<NamespaceDefinitionEnum.DESTINATION: 'destination'>, namespace_format=None, non_breaking_schema_updates_behavior=<NonBreakingSchemaUpdatesBehaviorEnu

### Get Sources

In [11]:
source_resp = client.sources.get_source(api.GetSourceRequest(source_id=connections[0].source_id))

In [12]:
source = source_resp.source_response

In [13]:
print(source)

SourceResponse(configuration=SourceAirtable(credentials=None, SOURCE_TYPE=<SourceAirtableAirtable.AIRTABLE: 'airtable'>), created_at=1768422088, definition_id='0efee448-6948-49e2-b786-17db50647908', name='hackernews-rss-newest', source_id='d73fb160-9fbd-457e-891e-22d5ac5b5aee', source_type='rss', workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', resource_allocation=None)


### Destinations

In [14]:
dest_resp = client.destinations.get_destination(api.GetDestinationRequest(connections[0].destination_id))

In [15]:
dest = dest_resp.destination_response

In [16]:
print(dest)

DestinationResponse(configuration=DestinationBigquery(dataset_id='hackernews_raw_data', dataset_location=<DatasetLocation.US_CENTRAL1: 'us-central1'>, project_id='iobruno-gcp-labs', cdc_deletion_mode=<CDCDeletionMode.HARD_DELETE: 'Hard delete'>, credentials_json='**********', DESTINATION_TYPE=<Bigquery.BIGQUERY: 'bigquery'>, disable_type_dedupe=False, loading_method=BatchedStandardInserts(additional_properties=None, method=<Method.STANDARD: 'Standard'>), raw_data_dataset=None), created_at=1768458307, definition_id='22f6c74f-5699-40ff-833c-4a879ea40133', destination_id='14e21b21-85d5-46b8-916a-df3c087ac4fe', destination_type='bigquery', name='bigquery-hackernews', workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', resource_allocation=None)
